In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as mse,r2_score as r2, mean_absolute_error as mae , make_scorer 
import sklearn.preprocessing as pre
from sklearn.model_selection import GridSearchCV, cross_val_score, TimeSeriesSplit
#Linear Models
import sklearn.linear_model as lm
#Trees - Non-Linear
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.ensemble import AdaBoostRegressor as ada

#NeuralNet - Piecewise Linear
from sklearn.neural_network import MLPRegressor as fnn

from main import RollingWindowSplit

### Estimate Model and Store Results

In [2]:
feature_set = [[32,53,'vix20',21],[22,63,'vix40',41],[12,73,'vix60',61],[2,83,'vix80',81]]
naif = {81:0.05,61:0.08,41:0.11,21:0.15}
purge = 5
alphaset = [0.0001,0.001,0.01,0.1,1,10,100,1000,10000]
maxdepth = {0.0001:3,0.001:5,0.01:10,0.1:15,1:20,10:40,100:80,1000:100,10000:1000}


In [5]:
def regression_test(path,intercept=True):
    surface = pd.read_csv(path)
    #Restrict database in order to eliminate some outliers concentrated before 2000   940:
    surface = surface.loc[300:3800]
    surface.reset_index(inplace=True) #Need to reset index cause after slicing index start from 940 and .loc wrongly indenty it
    surface.drop(columns='level_0',inplace=True)
    cv1 = pd.DataFrame(columns=['alpha','num_opt','VIX2','VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN'])
    cv2 = pd.DataFrame(columns=['alpha','num_opt','VIX2','VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN'])
    
    for alpha in alphaset:
        result1 = pd.DataFrame(columns=['alpha','num_opt','VIX2','VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN'],index=[21,41,61,81])
        result2 = pd.DataFrame(columns=['alpha','num_opt','VIX2','VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN'],index=[21,41,61,81])
    
        
#        '''LINEAR REGRESSION'''
#    
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#          
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#
#
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
#
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize Linear Regressor
#                regressor1 = lm.LinearRegression(fit_intercept=intercept)
#
#                #Fit the Linear Regressor
#                regressor1.fit(X80_train,Y1_train)
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(regressor1.predict(X80_test))
#
#
#                '''REGRESSION 2'''
#                #Intialize Linear Regressor
#                regressor2 = lm.LinearRegression(fit_intercept=intercept)
#
#                #Fit the Linear Regressor
#                regressor2.fit(X80_train,Y2_80_train)
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(regressor2.predict(X80_test) + VIX_test)
#
#               
#            #Store results in the Results Dataframe
#            result1.loc[result_index,'Linear'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'Linear'] = r2(Y2_true,Y2_hat)
#
#           
#
#        '''RIDGE REGRESSION'''
#
#
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#           
#
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30,purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values 
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize Ridge Regressor
#                regressor1 = lm.Ridge(alpha=alpha,fit_intercept=intercept,max_iter=2000)
#
#                #Fit the Ridge Regressor
#                regressor1.fit(X80_train,Y1_train)
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(regressor1.predict(X80_test))
#
#
#                '''REGRESSION 2'''
#                #Intialize Ridge Regressor
#                regressor2 = lm.Ridge(alpha=alpha,fit_intercept=intercept,max_iter=2000)
#
#                #Fit the Ridge Regressor
#                regressor2.fit(X80_train,Y2_80_train)
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
#
#              
#            result1.loc[result_index,'Ridge'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'Ridge'] = r2(Y2_true,Y2_hat)
#
#           
#        '''VIX80'''
#
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#
#
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize VIX Regressor
#                #regressor1 = lm.LinearRegression()
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(VIX_test)
#
#
#                '''REGRESSION 2'''
#                #Intialize VIX Regressor
#                #regressor2 = lm.LinearRegression()
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(VIX_test)
#
#            #Store results in the Results Dataframe
#            result1.loc[result_index,'VIX*2'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'VIX*2'] = r2(Y2_true,Y2_hat)
#
#        '''VIX Naif'''
#
#
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#
#
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values 
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize VIX Regressor
#                #regressor1 = lm.LinearRegression()
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(VIX_test*(1+naif[result_index]))
#
#
#                '''REGRESSION 2'''
#                #Intialize VIX Regressor
#                #regressor2 = lm.LinearRegression()
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(VIX_test*(1+naif[result_index]))
#
#            #Store results in the Results Dataframe
#            result1.loc[result_index,'VIX_Naif'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'VIX_Naif'] = r2(Y2_true,Y2_hat)
#
#
#
#        '''HUBER REGRESSION'''
#
#
#
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#           
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize Huber Regressor
#                regressor1 = lm.HuberRegressor(alpha=alpha,fit_intercept=intercept,max_iter=400)
#
#                #Fit the Huber Regressor
#                regressor1.fit(X80_train,Y1_train)
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(regressor1.predict(X80_test))
#
#
#                '''REGRESSION 2'''
#                #Intialize Huber Regressor
#                regressor2 = lm.HuberRegressor(alpha=alpha,fit_intercept=intercept,max_iter=400)
#
#                #Fit the Huber Regressor
#                regressor2.fit(X80_train,Y2_80_train)
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
#
#                
#            result1.loc[result_index,'Huber'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'Huber'] = r2(Y2_true,Y2_hat)
#
#          
#
#
#        '''LASSO REGRESSION'''
#
#
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#        
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30,purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values 
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize Lasso Regressor
#                regressor1 = lm.Lasso(alpha=alpha,fit_intercept=intercept,max_iter=2000)
#
#                #Fit the Lasso Regressor
#                regressor1.fit(X80_train,Y1_train)
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(regressor1.predict(X80_test))
#
#
#                '''REGRESSION 2'''
#                #Intialize Lasso Regressor
#                regressor2 = lm.Lasso(alpha=alpha,fit_intercept=intercept,max_iter=2000)
#
#                #Fit the Lasso Regressor
#                regressor2.fit(X80_train,Y2_80_train)
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
#
#
#            result1.loc[result_index,'Lasso'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'Lasso'] = r2(Y2_true,Y2_hat)
#
#            
#
#        '''ELASTIC NET'''
#
#        for fset in feature_set:
#            feature_start , feature_end , vix_code, result_index = fset
#            
#            Y1_true = []
#            Y1_hat  = []
#            Y2_true = []
#            Y2_hat  = []
#
#
#            for i,j in RollingWindowSplit().split_bounds(surface,1000,30,purge=purge):
#                train_start, train_end = i[0], i[1]
#                test_start, test_end = j[0], j[1]
#                #Split Train Set
#                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
#                Y1_train= surface.loc[train_start:train_end-1,'var30'].values 
#                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
#                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
#                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()
#
#
#                #Split Test Set
#                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
#                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
#                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
#                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test
#
#
#
#                '''REGRESSION 1'''
#                #Intialize Elastic Regressor
#                regressor1 = lm.ElasticNet(alpha=alpha,fit_intercept=intercept,max_iter=2000)
#
#                #Fit the Elastic Regressor
#                regressor1.fit(X80_train,Y1_train)
#
#                #Predict OOS
#                Y1_true.extend(Y1_test)
#                Y1_hat.extend(regressor1.predict(X80_test))
#
#
#                '''REGRESSION 2'''
#                #Intialize Elastic Regressor
#                regressor2 = lm.ElasticNet(alpha=alpha,fit_intercept=intercept,max_iter=2000)
#
#                #Fit the Elastic Regressor
#                regressor2.fit(X80_train,Y2_80_train)
#
#                #Predict OOS
#                Y2_true.extend(Y2_80_test  +  VIX_test)
#                Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)
#
#
#            result1.loc[result_index,'Elastic'] = r2(Y1_true,Y1_hat)
#            result2.loc[result_index,'Elastic'] = r2(Y2_true,Y2_hat)
            
            
            
        '''FEED-FORWARD NEURAL NET REGRESSION'''

        for fset in feature_set:
            feature_start , feature_end , vix_code, result_index = fset
            Y1_true = []
            Y1_hat  = []
            Y2_true = []
            Y2_hat  = []


            for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
                train_start, train_end = i[0], i[1]
                test_start, test_end = j[0], j[1]
                #Split Train Set
                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
                Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


                #Split Test Set
                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



                '''REGRESSION 1'''
                #Intialize FNN Regressor
                regressor1 = fnn(hidden_layer_sizes=(result_index,),alpha=alpha,max_iter=2000,early_stopping=False,shuffle=False,verbose=False)

                #Fit the FNN Regressor
                regressor1.fit(X80_train,Y1_train)

                #Predict OOS
                Y1_true.extend(Y1_test)
                Y1_hat.extend(regressor1.predict(X80_test))


                '''REGRESSION 2'''
                #Intialize FNN Regressor
                regressor2 = fnn(hidden_layer_sizes=(result_index,),alpha=alpha,max_iter=2000,early_stopping=False,shuffle=False,verbose=False)

                #Fit the FNN Regressor
                regressor2.fit(X80_train,Y2_80_train)

                #Predict OOS
                Y2_true.extend(Y2_80_test  +  VIX_test)
                Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)

            result1.loc[result_index,'FNN'] = r2(Y1_true,Y1_hat)
            result2.loc[result_index,'FNN'] = r2(Y2_true,Y2_hat)


        '''RF REGRESSION'''


        for fset in feature_set:
            feature_start , feature_end , vix_code, result_index = fset
            Y1_true = []
            Y1_hat  = []
            Y2_true = []
            Y2_hat  = []


            for i,j in RollingWindowSplit().split_bounds(surface,1000,30, purge=purge):
                train_start, train_end = i[0], i[1]
                test_start, test_end = j[0], j[1]
                #Split Train Set
                X80_train = surface.iloc[train_start:train_end,range(feature_start,feature_end,1)].values
                Y1_train= surface.loc[train_start:train_end-1,'var30'].values   
                VIX_train = surface.loc[train_start:train_end-1,vix_code].values
                Y2_80_train= surface.loc[train_start:train_end-1,'var30'].values - VIX_train
                #We used train_end -1 because .loc() is inclusive of the up bound contrary to .iloc() or range()


                #Split Test Set
                X80_test = surface.iloc[test_start:test_end,range(feature_start,feature_end,1)].values
                Y1_test= surface.loc[test_start:test_end-1,'var30'].values
                VIX_test = surface.loc[test_start:test_end-1,vix_code].values
                Y2_80_test= surface.loc[test_start:test_end-1,'var30'].values - VIX_test



                '''REGRESSION 1'''
                #Intialize FNN Regressor
                regressor1 = rf(max_depth=maxdepth[alpha],n_estimators=100)

                #Fit the FNN Regressor
                regressor1.fit(X80_train,Y1_train)

                #Predict OOS
                Y1_true.extend(Y1_test)
                Y1_hat.extend(regressor1.predict(X80_test))


                '''REGRESSION 2'''
                #Intialize FNN Regressor
                regressor2 = rf(max_depth=maxdepth[alpha],n_estimators=100)

                #Fit the FNN Regressor
                regressor2.fit(X80_train,Y2_80_train)

                #Predict OOS
                Y2_true.extend(Y2_80_test  +  VIX_test)
                Y2_hat.extend(regressor2.predict(X80_test) +  VIX_test)

            result1.loc[result_index,'RF'] = r2(Y1_true,Y1_hat)
            result2.loc[result_index,'RF'] = r2(Y2_true,Y2_hat)
            
            
            
            result1.loc[result_index,'alpha'] = alpha
            result2.loc[result_index,'alpha'] = alpha
            result1.loc[result_index,'num_opt'] = result_index
            result2.loc[result_index,'num_opt'] = result_index
        
        print(alpha)    
        result1.reset_index(drop=True,inplace=True)
        cv1 = pd.concat([cv1,result1])
        result2.reset_index(drop=True,inplace=True)
        cv2 = pd.concat([cv2,result2])
             
        
        
    return cv1, cv2

In [ ]:
result1, result2 = regression_test(intercept=True, path='/home/student/surface/Realized Variance/DatasetSPX/Dataset30_withmean.csv')

0.0001
0.001
0.01
0.1
1


In [186]:
cv_result2 = pd.DataFrame(columns=['VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN'],index=[21,41,61,81])
result2.reset_index(inplace=True,drop=True)
for index in cv_result2.index:
    for model in ['VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN']:
        line = result2[result2[model]==result2[(result2.num_opt==index)][[model]].max().values[0]]
        cv_result2.loc[index,model] = [line.alpha.values[0],np.round(line[model].values[0],3)]
    

In [184]:
cv_result1 = pd.DataFrame(columns=['VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN'],index=[21,41,61,81])
result1.reset_index(inplace=True,drop=True)
for index in cv_result1.index:
    for model in ['VIX_Naif','VIX*2','Linear','Ridge','Lasso','Elastic','Huber','RF','FNN']:
        line = result1[result1[model]==result1[(result1.num_opt==index)][[model]].max().values[0]]
        cv_result1.loc[index,model] = [line.alpha.values[0],np.round(line[model].values[0],3)]
    

In [187]:
cv_result1

,VIX_Naif,VIX*2,Linear,Ridge,Lasso,Elastic,Huber
21,"[0.0001, 0.17]","[0.0001, 0.12]","[0.0001, 0.257]","[100, 0.289]","[0.0001, 0.275]","[0.0001, 0.274]","[10000, 0.329]"
41,"[0.0001, 0.31]","[0.0001, 0.272]","[0.0001, 0.105]","[10000, 0.276]","[0.0001, 0.228]","[0.0001, 0.232]","[10000, 0.282]"
61,"[0.0001, 0.363]","[0.0001, 0.337]","[0.0001, 0.053]","[10000, 0.264]","[0.0001, 0.228]","[0.0001, 0.203]","[10000, 0.238]"
81,"[0.0001, 0.385]","[0.0001, 0.37]","[0.0001, -0.005]","[10000, 0.245]","[0.0001, 0.21]","[0.001, 0.184]","[10000, 0.226]"


In [188]:
cv_result2

,VIX_Naif,VIX*2,Linear,Ridge,Lasso,Elastic,Huber
21,"[0.0001, 0.17]","[0.0001, 0.12]","[0.0001, 0.257]","[100, 0.289]","[0.0001, 0.265]","[0.0001, 0.268]","[10000, 0.329]"
41,"[0.0001, 0.31]","[0.0001, 0.272]","[0.0001, 0.105]","[10000, 0.295]","[0.01, 0.313]","[0.01, 0.313]","[10000, 0.282]"
61,"[0.0001, 0.363]","[0.0001, 0.337]","[0.0001, 0.053]","[10000, 0.278]","[0.01, 0.36]","[0.01, 0.36]","[10000, 0.238]"
81,"[0.0001, 0.385]","[0.0001, 0.37]","[0.0001, -0.005]","[10000, 0.256]","[0.01, 0.384]","[0.01, 0.384]","[10000, 0.226]"
